# GMST-Py1812: Complete Pipeline Demonstration

This notebook demonstrates all 5 phases of the ITU-R P.1812-6 radio propagation prediction pipeline.

## Phases Overview
- **Phase 0**: Setup - Initialize environment, load and validate configuration
- **Phase 1**: Data Preparation - Download and cache land cover data from Sentinel Hub
- **Phase 2**: Batch Point Generation - Create receiver grid using point_generation utilities
- **Phase 3**: Batch Data Extraction - Extract elevation and land cover at receiver points
- **Phase 4**: Formatting & Export - Format profiles and validate CSV output
- **Phase 5**: P.1812 Analysis - Calculate propagation loss and field strength

Each phase demonstrates key functions and modules from the pipeline.

# Phase 0: Setup & Configuration

Initialize environment and load validated configuration.

## Imports

In [ ]:
import os
import math
import sys
import time
import csv
import json
import ast
import glob
from pathlib import Path
from dataclasses import dataclass
from typing import Iterable, Union, List, Tuple, Dict, Any

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import rasterio
from rasterio.transform import rowcol

import requests
import matplotlib.pyplot as plt

import Py1812.P1812

print("✓ All imports successful")

## Path Setup

In [ ]:
# Find and add project root to path
possible_roots = [
    Path.cwd(),
    Path.cwd().parent,
    Path.cwd().parent.parent,
]

project_root = None
for root in possible_roots:
    if (root / 'src' / 'pipeline').exists() or (root / 'config_example.json').exists():
        project_root = root
        break

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    sys.path.insert(0, str(project_root / 'src'))

# Try to import Sentinel Hub credentials
try:
    from config_sentinel_hub import (
        SH_CLIENT_ID, SH_CLIENT_SECRET,
        TOKEN_URL, PROCESS_URL, COLLECTION_ID,
    )
except ImportError:
    SH_CLIENT_ID = ''
    SH_CLIENT_SECRET = ''
    TOKEN_URL = ''
    PROCESS_URL = ''
    COLLECTION_ID = ''

# Define all data paths
profiles_dir = project_root / 'data' / 'profiles'
landcover_dir = project_root / 'data' / 'landcover'
output_dir = project_root / 'data' / 'output'
srtm_dir = project_root / 'data' / 'srtm'
brzones_dir = project_root / 'data' / 'brzones'

# Create directories
for d in [profiles_dir, landcover_dir, output_dir, srtm_dir, brzones_dir]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print('✓ All data directories created')

## Configuration Management

Load, validate, and extract configuration using ConfigManager.

In [ ]:
# Import configuration management utilities
from pipeline.config import (
    ConfigManager,
    _load_sentinel_hub_credentials,
    get_transmitter_info,
    get_p1812_params,
    get_receiver_generation_params,
    get_land_cover_mappings,
)

# Load and validate configuration
config_path = project_root / 'config_example.json'
if not config_path.exists():
    raise FileNotFoundError(f'Config file not found: {config_path}')

config_mgr = ConfigManager.from_file(config_path)
CONFIG = config_mgr.config

# IMPORTANT: Load Sentinel Hub credentials from config_sentinel_hub.py
CONFIG = _load_sentinel_hub_credentials(CONFIG)

print(f'✓ Loaded and validated configuration from {config_path.name}')
print(f'✓ Sentinel Hub credentials loaded from config_sentinel_hub.py')

# Extract configuration sections
tx_info = get_transmitter_info(CONFIG)
p1812_params = get_p1812_params(CONFIG)
rx_gen_params = get_receiver_generation_params(CONFIG)

print(f'\nTransmitter:')
for key, val in tx_info.items():
    print(f'  {key}: {val}')

print(f'\nP.1812 Parameters:')
for key, val in p1812_params.items():
    print(f'  {key}: {val}')

print(f'\nReceiver Generation:')
print(f'  Max distance: {rx_gen_params["max_distance_km"]} km')
print(f'  Azimuth step: {rx_gen_params["azimuth_step"]}°')
print(f'  Distance step: {rx_gen_params["distance_step"]} km')

# Derived values
tx_lon = tx_info['longitude']
tx_lat = tx_info['latitude']
max_distance_km = rx_gen_params['max_distance_km']
azimuths = list(range(0, 360, rx_gen_params['azimuth_step']))
distances = np.arange(
    rx_gen_params['distance_step'],
    max_distance_km + rx_gen_params['distance_step'],
    rx_gen_params['distance_step']
)

print(f'\nDerived:')
print(f'  Azimuths: {len(azimuths)} steps')
print(f'  Distances: {len(distances)} steps')
print(f'  Total receiver points: {len(azimuths) * len(distances)}')


## Transmitter Definition

Create transmitter using actual pipeline Transmitter class.

In [ ]:
from pipeline.point_generation import Transmitter

tx = Transmitter(
    tx_id=tx_info['tx_id'],
    lon=tx_info['longitude'],
    lat=tx_info['latitude'],
    htg=tx_info['antenna_height_tx'],
    hrg=tx_info['antenna_height_rx'],
    f=p1812_params['frequency_ghz'],
    pol=p1812_params['polarization'],
    p=p1812_params['time_percentage'],
)

print(f"✓ Transmitter created:")
print(f"  {tx}")

## Elevation Data Initialization

Initialize and cache SRTM elevation data.

In [ ]:
from propagation.profile_extraction import set_srtm_cache_dir, _get_srtm_data

print("\nInitializing SRTM elevation data...")
init_start = time.time()

try:
    # Set SRTM cache directory
    set_srtm_cache_dir(str(srtm_dir))
    
    # Initialize SRTM data handler
    srtm_data = _get_srtm_data()
    
    # Pre-download tile
    print(f"  Downloading SRTM1 tile for TX area ({tx_lat}, {tx_lon})...")
    tx_elev = srtm_data.get_elevation(tx_lat, tx_lon)
    
    init_time = time.time() - init_start
    print(f"✓ SRTM elevation data ready ({init_time:.2f}s)")
    print(f"  TX elevation: {tx_elev}m")
    
    # Load HGT tile into memory
    hgt_files = glob.glob(str(srtm_dir / "*.hgt"))
    if hgt_files:
        hgt_path = hgt_files[0]
        print(f"\n  Loading HGT tile into memory...")
        with rasterio.open(hgt_path) as dem_src:
            dem_band_data = dem_src.read(1)
            dem_transform = dem_src.transform
        print(f"  ✓ HGT loaded: {dem_band_data.shape} array")
    else:
        dem_band_data = None
        dem_transform = None
        
except Exception as e:
    print(f"✗ Error initializing SRTM: {e}")
    srtm_data = None
    dem_band_data = None
    dem_transform = None

# Phase 1: Data Preparation

Land cover download using Sentinel Hub utilities.

In [ ]:
from pipeline.data_preparation import prepare_landcover
from propagation.profile_extraction import resolve_credentials

print("\nPhase 1: Data Preparation")
print("="*60)

# Get Sentinel Hub config from CONFIG (now with credentials loaded)
sh_config = CONFIG.get('SENTINEL_HUB', {})
client_id = sh_config.get('client_id', '').strip()
client_secret = sh_config.get('client_secret', '').strip()
collection_id = sh_config.get('collection_id', '').strip()
token_url = sh_config.get('token_url', '')
process_url = sh_config.get('process_url', '')

print(f"\nSentinel Hub Configuration:")
print(f"  Has client_id: {bool(client_id)}")
print(f"  Has client_secret: {bool(client_secret)}")
print(f"  Has collection_id: {bool(collection_id)}")

if client_id and client_secret and collection_id:
    print(f"\nAttempting land cover download...")
    print(f"  Location: ({tx_lat}, {tx_lon})")
    print(f"  Collection ID: {collection_id[:8]}... (truncated)")
    print(f"  Save to: {landcover_dir}")
    
    try:
        start = time.time()
        lc_path = prepare_landcover(
            lat=tx_lat,
            lon=tx_lon,
            cache_dir=landcover_dir,
            client_id=client_id,
            client_secret=client_secret,
            token_url=token_url,
            process_url=process_url,
            collection_id=collection_id,
            year=sh_config.get('year', 2020),
            buffer_m=sh_config.get('buffer_m', 11000),
            chip_px=sh_config.get('chip_px', 734),
            verbose=True,
        )
        elapsed = time.time() - start
        print(f"\n✓ Land cover preparation complete ({elapsed:.1f}s)")
        print(f"  Saved to: {lc_path.name}")
        print(f"  File size: {lc_path.stat().st_size / (1024*1024):.1f} MB")
        phase1_landcover_path = lc_path
    except Exception as e:
        error_msg = str(e)[:200]
        print(f"\n✗ Download failed: {error_msg}")
        phase1_landcover_path = None
else:
    print(f"\n✗ Missing Sentinel Hub configuration")
    if not client_id:
        print(f"  - client_id is empty")
    if not client_secret:
        print(f"  - client_secret is empty")
    if not collection_id:
        print(f"  - collection_id is empty")
    phase1_landcover_path = None

print(f"\nPhase 1 Status: {'✓ Complete' if phase1_landcover_path else '✗ Failed'}")


# Phase 2: Batch Point Generation

Generate receiver points using point_generation utilities.

In [ ]:
from pipeline.point_generation import Transmitter, generate_receiver_grid

print("\nPhase 2: Batch Point Generation")
print("="*60)

# Create Transmitter object exactly as the real pipeline does
transmitter = Transmitter(
    tx_id=tx_info['tx_id'],
    lon=tx_info['longitude'],
    lat=tx_info['latitude'],
    htg=tx_info['antenna_height_tx'],
    f=p1812_params['frequency_ghz'],
    pol=p1812_params['polarization'],
    p=p1812_params['time_percentage'],
    hrg=tx_info['antenna_height_rx'],
)

# Calculate number of azimuths from azimuth_step
num_azimuths = int(360 / rx_gen_params['azimuth_step'])

print(f"\nGenerating receiver grid...")
print(f"  Max distance: {rx_gen_params['max_distance_km']} km")
print(f"  Distance step: {rx_gen_params['distance_step']} km")
print(f"  Azimuth step: {rx_gen_params['azimuth_step']}°")
print(f"  Number of azimuths: {num_azimuths}")

start = time.time()
receivers_gdf = generate_receiver_grid(
    tx=transmitter,
    max_distance_km=rx_gen_params['max_distance_km'],
    sampling_resolution_m=rx_gen_params['sampling_resolution'],
    num_azimuths=num_azimuths,
    include_tx_point=True,
)
elapsed = time.time() - start

print(f"\n✓ Generated {len(receivers_gdf)} receiver points in {elapsed:.3f}s")
print(f"\nGeoDataFrame structure:")
print(f"  CRS: {receivers_gdf.crs}")
print(f"  Columns: {list(receivers_gdf.columns)}")
print(f"  Distance range: {receivers_gdf['distance_km'].min():.2f}-{receivers_gdf['distance_km'].max():.2f} km")


# Phase 3: Batch Data Extraction

Extract elevation and land cover at receiver points.

In [ ]:
from pipeline.data_extraction import extract_data_for_receivers, RasterPreloader, map_landcover_codes
from pathlib import Path

print("\nPhase 3: Batch Data Extraction")
print("="*60)

# Prepare for real data extraction
print(f"\nPreparing for batch data extraction...")
print(f"  Total receiver points: {len(receivers_gdf)}")

# Get land cover mappings from config
lcm10_to_ct = CONFIG.get('LCM10_TO_CT', {str(k): v for k, v in enumerate([2]*255)})
ct_to_r = CONFIG.get('CT_TO_R', {1: 0, 2: 10, 3: 50, 4: 100, 5: 200})

# Try to locate cached landcover data (from Phase 1)\n",
tx_lat = tx_info['latitude']
tx_lon = tx_info['longitude']
sh_config = CONFIG.get('SENTINEL_HUB', {})
landcover_pattern = f"lcm10_{tx_lat}_{tx_lon}*.tif"

# Check if landcover exists in cache
import glob
cached_landcover = glob.glob(str(landcover_dir / landcover_pattern))
landcover_path = Path(cached_landcover[0]) if cached_landcover else None

if landcover_path and landcover_path.exists():
    print(f"✓ Found cached landcover: {landcover_path.name}")
else:
    print(f"⚠ No cached landcover found")
    print(f"  To enable land cover extraction:")
    print(f"  1. Configure Sentinel Hub credentials in config_sentinel_hub.py")
    print(f"  2. Run Phase 1 first, or")
    print(f"  3. Download manually and place in {landcover_dir}")
    landcover_path = None

# Prepare DEM path
dem_path = srtm_dir / 'SRTM1.vrt'
if not dem_path.exists():
    # Try to find any HGT files
    hgt_files = glob.glob(str(srtm_dir / '*.hgt'))
    if hgt_files:
        dem_path = Path(hgt_files[0])
        print(f"✓ Found SRTM HGT file: {dem_path.name}")
    else:
        dem_path = None
        print(f"⚠ No DEM data found in {srtm_dir}")

# Prepare zones path
zones_path = brzones_dir / 'zones_map_BR.json'
if not zones_path.exists():
    print(f"⚠ Zones GeoJSON not found at {zones_path}")
    zones_path = None

# Perform batch data extraction if we have the data
print(f"\nBatch data extraction configuration:")
print(f"  DEM: {dem_path.name if dem_path else 'Not available'}")
print(f"  Landcover: {landcover_path.name if landcover_path else 'Not available'}")
print(f"  Zones: {zones_path.name if zones_path else 'Not available'}")

# Always run extraction, even with missing data (graceful fallback)
try:
    enriched_gdf = extract_data_for_receivers(
        receivers_gdf=receivers_gdf.copy(),
        dem_path=dem_path or Path('/tmp/dummy.vrt'),
        landcover_path=landcover_path or Path('/tmp/dummy.tif'),
        zones_path=zones_path or Path('/tmp/dummy.json'),
        lcm10_to_ct=lcm10_to_ct,
        ct_to_r=ct_to_r,
        verbose=True,
    )
    receivers_gdf = enriched_gdf
    extraction_success = True
except Exception as e:
    # Fallback: manually add default values\n",
    print(f"⚠ Data extraction failed: {str(e)[:100]}...")
    print(f"\nApplying default values as fallback...")
    
    # Extract elevation from loaded HGT if available
    if dem_band_data is not None:
        elevations = []
        for idx, row in receivers_gdf.iterrows():
            lon, lat = row.geometry.x, row.geometry.y
            try:
                col, row_idx = rowcol(dem_transform, lon, lat)
                if 0 <= col < dem_band_data.shape[1] and 0 <= row_idx < dem_band_data.shape[0]:
                    elev = dem_band_data[row_idx, col]
                    elevations.append(float(elev) if elev > -32000 else 0.0)
                else:
                    elevations.append(0.0)
            except:
                elevations.append(0.0)
        receivers_gdf['h'] = elevations
    else:
        receivers_gdf['h'] = 0.0
    
    # Default land cover values
    receivers_gdf['ct'] = 254  # No-data
    receivers_gdf['Ct'] = 2    # Default category: vegetation
    receivers_gdf['R'] = 10.0  # Default resistance: 10 ohms
    receivers_gdf['zone'] = 4  # Default: Inland
    
    extraction_success = False
    print(f"✓ Applied defaults")

# Summary
print(f"\nPhase 3 Complete:")
print(f"  Status: {'✓ Full extraction' if extraction_success else '⚠ Fallback defaults'}")
print(f"  Receiver points: {len(receivers_gdf)}")
print(f"  Required columns: {all(col in receivers_gdf.columns for col in ['h', 'Ct', 'R', 'zone'])}")
print(f"\nData summary:")
print(receivers_gdf[['tx_id', 'rx_id', 'distance_km', 'h', 'Ct', 'R', 'zone']].head(10))
print(f"\nStatistics:")
print(f"  Elevation: {receivers_gdf['h'].min():.1f} - {receivers_gdf['h'].max():.1f} m")
print(f"  Categories: {dict(receivers_gdf['Ct'].value_counts().sort_index())}")
print(f"  Zones: {dict(receivers_gdf['zone'].value_counts().sort_index())}")

# Phase 4: Formatting & Export

Format profiles using ProfileFormatter and validate output.

In [ ]:
from pipeline.formatting import format_and_export_profiles

print("\nPhase 4: Formatting & Export")
print("="*60)

# Use the pipeline's format_and_export_profiles function
# This handles everything: formatting, validation, and CSV export
distance_step_km = rx_gen_params['distance_step']  # From config_example.json

csv_path = profiles_dir / "demo_profiles.csv"
start = time.time()
df_profiles, result_path = format_and_export_profiles(
    receivers_gdf=receivers_gdf,
    output_path=csv_path,
    frequency_ghz=p1812_params['frequency_ghz'],
    time_percentage=p1812_params['time_percentage'],
    polarization=p1812_params['polarization'],
    htg=tx_info['antenna_height_tx'],
    hrg=tx_info['antenna_height_rx'],
    distance_step_km=distance_step_km,
    verbose=True,
)
elapsed = time.time() - start

print(f"\nPhase 4 completed in {elapsed:.3f}s")
print(f"CSV path: {result_path}")


# Phase 5: P.1812 Analysis

Process profiles through P.1812 propagation model.

In [ ]:
from propagation.propagation_calculator import main as batch_p1812

print("\nPhase 5: P.1812 Batch Analysis")
print("="*60)

# Run batch P.1812 processor
output_dir = project_root / 'data' / 'output'
output_dir.mkdir(parents=True, exist_ok=True)

start = time.time()
result = batch_p1812(
    profiles_dir=profiles_dir,
    output_dir=output_dir,
)
elapsed = time.time() - start

print(f"\n✓ Phase 5 completed in {elapsed:.1f}s")
print(f"  Results saved to: {output_dir}")


# Utilities: Logging and Validation

Demonstrate logging utilities and validation functions.

In [ ]:
from utils.logging import Timer, ProgressTracker, print_header, print_success, print_warning, format_bytes, format_duration
from utils.validation import validate_config, validate_dataframe

print("\nUtilities Demonstration")
print("="*60)

# Timer context manager
print("\nTimer utility:")
with Timer("Example operation"):
    time.sleep(0.1)
    print("  Doing work...")

# Progress tracker
print("\nProgress tracker:")
tracker = ProgressTracker(total=3, name="Processing")
tracker.start()  # Must call start() before update()
for i in range(3):
    time.sleep(0.1)
    tracker.update(force=(i==2))  # Force final update
    print(f"  Step {i+1}: {(tracker.current/tracker.total)*100:.1f}% complete")
tracker.finish()

# Formatting utilities
print("\nFormatting utilities:")
try:
    from utils.logging import format_bytes, format_duration
    print(f"  Bytes: {format_bytes(1024*1024)} = 1 MB")
    print(f"  Duration: {format_duration(3661)} = 1 hour, 1 minute, 1 second")
except ImportError:
    print(f"  format_bytes(1048576) → 1.0 MB")
    print(f"  format_duration(3661) → 1 hour, 1 minute, 1 second")

# Validation
print("\nValidation:")
print("  CONFIG validation: ", end="")
try:
    validate_config(CONFIG)
    print("✓ Valid")
except Exception as e:
    print(f"✗ Invalid: {str(e)[:50]}")

print(f"\nGeoDataFrame validation: ", end="")
try:
    validate_dataframe(receivers_gdf, required_cols=['geometry'])
    print(f"✓ Valid ({len(receivers_gdf)} rows)")
except Exception as e:
    print(f"✗ Invalid: {e}")

print("\n✓ All utilities demonstrated")

# Complete Pipeline Summary

## Phases Completed
1. **Phase 0**: Configuration management with validation
2. **Phase 1**: Sentinel Hub data preparation
3. **Phase 2**: Receiver point generation (point_generation module)
4. **Phase 3**: Data extraction and validation
5. **Phase 4**: Profile formatting using ProfileFormatter
6. **Phase 5**: P.1812 propagation calculations

## Key Modules Demonstrated

### Pipeline Modules
- **pipeline.config**: ConfigManager, configuration extraction helpers
- **pipeline.point_generation**: Transmitter, receiver grid generation
- **pipeline.formatting**: ProfileFormatter, CSV validation
- **pipeline.data_extraction**: RasterPreloader, zone extraction
- **pipeline.orchestration**: PipelineOrchestrator (multi-phase execution)

### Propagation Modules
- **propagation.profile_extraction**: SRTM utilities, Sentinel Hub helpers
- **propagation.profile_parser**: Profile loading and parameter parsing
- **propagation.propagation_calculator**: Batch P.1812 processing

### Utility Modules
- **utils.logging**: Timer, ProgressTracker, formatting utilities
- **utils.validation**: Configuration, data, and output validation

## Performance Notes
- Phase 0-1: ~30-60 seconds
- Phase 2-3: ~5-10 seconds
- Phase 4: ~2-3 seconds
- Phase 5: ~30-60 seconds
- **Total**: ~2-3 minutes

## Next Steps
1. Configure Sentinel Hub credentials
2. Modify config_example.json for your location
3. Use PipelineOrchestrator for automated multi-phase execution
4. Visualize results using output CSV and GeoJSON files

# Phase 6: Results Visualization

Visualize pipeline results using interactive Plotly charts and deck.gl maps.

In [ ]:
# Enable inline Plotly display in Jupyter
import plotly.io as pio
pio.renderers.default = 'notebook'

from utils.visualization import (
    create_loss_distribution_chart,
    create_field_strength_chart,
    create_loss_vs_distance_scatter,
    create_azimuth_heatmap,
    create_receiver_map,
    create_statistics_summary,
    print_summary,
)

print('\nPhase 6: Results Visualization')
print('='*60)

# Load results
output_dir = project_root / 'data' / 'output'
results_files = list(output_dir.glob('results_*.csv'))

if results_files:
    latest_results = sorted(results_files, key=lambda p: p.stat().st_mtime)[-1]
    print(f'\nLoading results from: {latest_results.name}')
    
    results_df = pd.read_csv(latest_results)
    print(f'✓ Loaded {len(results_df)} profile results')
    
    # Generate statistics
    summary = create_statistics_summary(results_df)
    print_summary(summary)
    
    # Create visualizations
    print('\nGenerating interactive visualizations...')
    
    # Chart 1: Lb distribution
    print('\n1. Lb (Basic Transmission Loss) Distribution')
    fig_lb = create_loss_distribution_chart(results_df)
    if fig_lb:
        fig_lb.show()
    
    # Chart 2: Ep distribution
    print('\n2. Ep (Electric Field Strength) Distribution')
    fig_ep = create_field_strength_chart(results_df)
    if fig_ep:
        fig_ep.show()
    
    # Chart 3: Lb vs Distance scatter
    print('\n3. Lb vs Distance Scatter Plot (colored by azimuth)')
    fig_scatter = create_loss_vs_distance_scatter(results_df)
    if fig_scatter:
        fig_scatter.show()
    
    # Chart 4: Azimuth heatmap
    print('\n4. Azimuth-Distance Heatmap')
    fig_heatmap = create_azimuth_heatmap(results_df)
    if fig_heatmap:
        fig_heatmap.show()
    
    # Map: Receiver locations with results
    print('\n5. Interactive Map (deck.gl) - Receiver Points Colored by Loss')
    try:
        from pipeline.config import ConfigManager, get_transmitter_info, get_receiver_generation_params
        from pipeline.point_generation import Transmitter, generate_receiver_grid
        
        config_path = project_root / 'config_example.json'
        config_mgr = ConfigManager.from_file(config_path)
        CONFIG = config_mgr.config
        
        tx_info = get_transmitter_info(CONFIG)
        rx_gen_params = get_receiver_generation_params(CONFIG)
        
        tx = Transmitter(
            tx_id=tx_info['tx_id'],
            lon=tx_info['longitude'],
            lat=tx_info['latitude'],
            htg=tx_info['antenna_height_tx'],
            f=CONFIG['P1812']['frequency_ghz'],
            pol=CONFIG['P1812']['polarization'],
            p=CONFIG['P1812']['time_percentage'],
            hrg=tx_info['antenna_height_rx'],
        )
        
        receivers_gdf = generate_receiver_grid(
            tx=tx,
            max_distance_km=rx_gen_params['max_distance_km'],
            sampling_resolution_m=rx_gen_params['sampling_resolution'],
            num_azimuths=int(360 / rx_gen_params['azimuth_step']),
            include_tx_point=True,
        )
        
        output_map_path = project_root / 'data' / 'output' / 'receiver_map.html'
        deck_map, map_html_path = create_receiver_map(receivers_gdf, results_df, output_path=output_map_path)
        if deck_map:
            print(f'Map saved to: {map_html_path}')
        
    except Exception as e:
        print(f'Map error: {str(e)[:100]}')
    
    print('\nVisualization complete')
else:
    print('\nNo results found. Run Phase 5 first.')


# Summary

## Phases Executed
1. **Phase 0**: Configuration management with validation
2. **Phase 1**: Sentinel Hub data preparation (optional)
3. **Phase 2**: Receiver point generation
4. **Phase 3**: Data extraction (elevation, landcover, zones)
5. **Phase 4**: Profile formatting and CSV export
6. **Phase 5**: P.1812 batch processing and results CSV
7. **Phase 6**: Results visualization with Plotly and deck.gl

## Key Outputs
- **Profiles CSV**: `data/profiles/profiles_TX_*.csv` (1,512 profiles with distance arrays)
- **Results CSV**: `data/output/results_TX_*.csv` (Lb and Ep for each profile)
- **Visualizations**: Interactive charts and maps showing loss/field strength distribution

## Next Steps
1. Modify `config_example.json` for different locations
2. Run the full pipeline for your specific transmitter location
3. Analyze results using the provided visualizations
4. Export results for further analysis or reporting